# Generate tables for metrics per city, per year, and per parcel

In [1]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', 150)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
fulton_parcels = gpd.read_file('data/fulton_parcels.geojson')
appeals_all_years = pd.read_excel('data/appeals2011-2022.xlsx')
fulton_cities = gpd.read_file('data/fulton_cities.geojson')

We want to combine the geometries of Atlanta and the small 'city' to the left of it, named 'Fulton County', which only contains 'Fulton County Airport - Brown Field (FTY)Fulton County Airport - Brown Field (FTY)'.

In [3]:
# TODO chained indexing?
fulton_cities.loc[fulton_cities['Name'] == 'Atlanta', 'geometry'] = fulton_cities[
    fulton_cities['Name'].isin(['Atlanta', 'Fulton County'])
].unary_union
fulton_cities = fulton_cities[fulton_cities['Name'] != 'Fulton County']

In [4]:
# Quick data EDA and quality check including coordinate system for spatial join
print(f"CRS: {fulton_parcels.crs}")
print(f"CRS matches?: {fulton_parcels.crs == fulton_cities.crs}")
print(f"Appeals shape: {appeals_all_years.shape}")
print(f"Fulton parcels shape: {fulton_parcels.shape}")
print(f"Fulton cities shape: {fulton_cities.shape}")
print(fulton_parcels.columns)
print(appeals_all_years.columns)
print(fulton_cities.columns)
print(appeals_all_years['TAXYR'].unique())

CRS: EPSG:4326
CRS matches?: True
Appeals shape: (279045, 7)
Fulton parcels shape: (367584, 29)
Fulton cities shape: (15, 11)
Index(['OBJECTID', 'ParcelID', 'TaxYear', 'Address', 'AddrNumber',
       'AddrPreDir', 'AddrStreet', 'AddrSuffix', 'AddrPosDir', 'AddrUntTyp',
       'AddrUnit', 'Owner', 'OwnerAddr1', 'OwnerAddr2', 'TaxDist', 'LUCode',
       'ClassCode', 'ExCode', 'LivUnits', 'LandAcres', 'NbrHood', 'Subdiv',
       'SubdivNum', 'SubdivLot', 'SubdivBlck', 'FeatureID', 'Shape__Area',
       'Shape__Length', 'geometry'],
      dtype='object')
Index(['TAXYR', 'PARID', 'HEARTYP', 'SUBKEY', 'NOTICE ASSESSMENT',
       'CURRENT ASSESSMENT', 'CLASS'],
      dtype='object')
Index(['OBJECTID', 'Name', 'TaxCode', 'AreaAc', 'AreaSqMi', 'GEOID10',
       'FeatureID', 'MuniCode', 'Shape__Area', 'Shape__Length', 'geometry'],
      dtype='object')
[2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022]


## Generate table for metrics by city

Taxyr range in appeals data is [2011, 2022]; taxyr is one year behind current year
because assessments get sent out early in the following year for the proceeding year

In [5]:
fulton_parcels.columns

Index(['OBJECTID', 'ParcelID', 'TaxYear', 'Address', 'AddrNumber',
       'AddrPreDir', 'AddrStreet', 'AddrSuffix', 'AddrPosDir', 'AddrUntTyp',
       'AddrUnit', 'Owner', 'OwnerAddr1', 'OwnerAddr2', 'TaxDist', 'LUCode',
       'ClassCode', 'ExCode', 'LivUnits', 'LandAcres', 'NbrHood', 'Subdiv',
       'SubdivNum', 'SubdivLot', 'SubdivBlck', 'FeatureID', 'Shape__Area',
       'Shape__Length', 'geometry'],
      dtype='object')

In [6]:
# Spatially join Fulton parcels to Fulton cities to get the city for each parcel
fulton_parcels = fulton_parcels.sjoin(fulton_cities[['Name', 'geometry']], how='left', predicate='within')
fulton_parcels['WKT'] = fulton_parcels.representative_point()
fulton_parcels = fulton_parcels.drop(columns=['geometry'])

Outer join appeals and parcel data on ParcelID, producing a df with all appeals
between 2011-2022 and their associated parcel data (in 2022), along with any parcels that did not have any appeals filed throughout the period. We want to check how many appeals do not have associated parcel info; this would indicate parcels have changed over the years and would raise concerns about this method.

In [7]:
appeals_w_parcel = fulton_parcels.merge(
    appeals_all_years,
    left_on='ParcelID',
    right_on='PARID',
    how='outer'
)
print(appeals_w_parcel.shape)

(514040, 38)


In [8]:
appeals_without_parcel_info = len(appeals_w_parcel[
    appeals_w_parcel['NOTICE ASSESSMENT'].notna()
    & appeals_w_parcel['Address'].isna()
])
rows_without_parcelid = len(appeals_w_parcel[
    appeals_w_parcel['ParcelID'].isna()
])

print(appeals_without_parcel_info, appeals_without_parcel_info == rows_without_parcelid)

6381 True


6.4K parcels have had their IDs changed over the years. Parcel change is quite rare and is irrelevant to many of our questions (e.g. how many times did a parcel file appeals between 2011-2022; if the parcel no longer exists, that question is difficult to answer). Further, retaining these appeals without associated parcel data will skew some other calculations. For instance, calculating what percent of parcels out of the total pool filed an appeal at least once. The count of parcels filing an appeal will be inflated and the count of parcels will be reduced.

Since only about 2% of appeals are impacted, these will be dropped. All further metrics will be highly accurate but not perfectly exact calculations.

In [9]:
appeals_w_parcel = appeals_w_parcel[
    appeals_w_parcel['ParcelID'].notna()
]

count_appeals = len(appeals_w_parcel[appeals_w_parcel['NOTICE ASSESSMENT'].notna()])
print("Raw count of total appeals, including repeat appeals: ", count_appeals)

Raw count of total appeals, including repeat appeals:  272664


Generate a table for appeal metrics by city within Fulton County. See documentation (metrics by city) for further details

In [10]:
# Rename column from 'Name' to 'City'
appeals_w_parcel.rename(columns={'Name': 'City'}, inplace=True)

# Create a DF with only appeals and their associated parcel data
all_appeals = appeals_w_parcel[appeals_w_parcel['NOTICE ASSESSMENT'].notna()]

# Calculate number of unique parcels in entire county as a reference
count_unique_parcels = len(appeals_w_parcel.drop_duplicates('ParcelID'))

# Count of appeals filed in each city
count_appeals_city = all_appeals.groupby('City')['ParcelID'].count()

# Count of unique parcels in each city
count_parcels_city = appeals_w_parcel.drop_duplicates(
    subset='ParcelID'
).groupby('City')['ParcelID'].count()

# Percent of total parcels (Fulton County) in each city
pct_parcels_in_city = count_parcels_city / count_unique_parcels * 100

# Percent of overall appeals filed by city
pct_appeals_in_city = count_appeals_city / len(all_appeals) * 100

# Normalized metric
pct_appeals_over_pct_parcels = pct_appeals_in_city / pct_parcels_in_city

# Count of parcels filing an appeal at least once
count_unique_parcels_filing_appeal_city = all_appeals.drop_duplicates(
    subset='ParcelID'
).groupby('City')['ParcelID'].count()

# Percent of parcels filing an appeal at least once (percent of the entire Fulton county)
pct_filing_appeal_of_fulton = count_unique_parcels_filing_appeal_city / count_unique_parcels * 100

# Percent of parcels filing an appeal at least once by city
pct_filing_appeal_by_city = count_unique_parcels_filing_appeal_city / count_parcels_city * 100

# Create a DF with only successfull appeals and their associated parcel data, including
# the difference between the notice assessment and the current assessment (appeal change)
all_appeals['diff'] = all_appeals['NOTICE ASSESSMENT'] - all_appeals['CURRENT ASSESSMENT']
all_appeals['pct_diff'] = all_appeals['diff'] / all_appeals['NOTICE ASSESSMENT'] * 100
successful_appeals = all_appeals[all_appeals['diff'] > 0]

# Adjust diff for inflation, using CPI-U from the
# BLS: https://data.bls.gov/timeseries/CUUR0000SA0
# Use December CPI for each year as demoninator, and
# 2022 December CPI as numerator
december_2022_cpi = 296.797
december_cpi_year = {
    2011: 225.672,
    2012: 229.601,
    2013: 233.049,
    2014: 234.812,
    2015: 236.525,
    2016: 241.432,
    2017: 246.524,
    2018: 251.233,
    2019: 256.974,
    2020: 260.474,
    2021: 278.802,
    2022: 296.797
}
inflation_scalar = {x: december_2022_cpi / december_cpi_year[x] for x in december_cpi_year}
print(f"Inflation scalars: {inflation_scalar}")

successful_appeals['diff'] = successful_appeals.apply(
    lambda row: row['diff'] / inflation_scalar[int(row['TAXYR'])],
    axis=1
)

# Success count by city
success_count_city = successful_appeals.groupby('City')['ParcelID'].count()

# Success rate of appeals filed by city
success_rate_city = success_count_city / count_appeals_city * 100

# Average change in assessment by city
avg_appeal_change_city = successful_appeals.groupby('City')['diff'].agg('mean')
pct_avg_appeal_change_city = successful_appeals.groupby('City')['pct_diff'].agg('mean')
med_appeal_change_city = successful_appeals.groupby('City')['diff'].agg('median')

# Total sum appealed, adjusted for inflation
sum_appealed_city = successful_appeals.groupby('City')['diff'].agg('sum')

# Lost revenue, multiply 40% of assessed value by conservative 1.08% millage rate, or 0.00432
# TODO run this by someone
lost_revenue_city = successful_appeals.groupby('City')['diff'].agg('sum') * 0.00432

# Percent of Parcels Filing More than _[2,11]_ Appeals
num_appeals_by_parcel = all_appeals.groupby(['ParcelID', 'City'])['ParcelID'].count()

percent_filing_geq_than_n_appeals = {}
for i in range(1, 11):
    num_filing_more_than_n = num_appeals_by_parcel[num_appeals_by_parcel >= i].groupby('City').count()
    percent_filing_geq_than_n_appeals[i] =  num_filing_more_than_n / count_parcels_city * 100

# TODO Percent of overall taxbill reduction from appeals. Assessment data for all parcels for all
# years is needed to calculate this.

C:\Users\Nick\AppData\Local\Temp\ipykernel_4236\3528718426.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_appeals['diff'] = all_appeals['NOTICE ASSESSMENT'] - all_appeals['CURRENT ASSESSMENT']
C:\Users\Nick\AppData\Local\Temp\ipykernel_4236\3528718426.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_appeals['pct_diff'] = all_appeals['diff'] / all_appeals['NOTICE ASSESSMENT'] * 100


Inflation scalars: {2011: 1.3151698039632742, 2012: 1.2926642305564873, 2013: 1.2735390411458536, 2014: 1.2639771391581351, 2015: 1.25482295740408, 2016: 1.2293192286026708, 2017: 1.2039274066622319, 2018: 1.1813615249589027, 2019: 1.1549689851891634, 2020: 1.1394496187719314, 2021: 1.0645440133141082, 2022: 1.0}


C:\Users\Nick\AppData\Local\Temp\ipykernel_4236\3528718426.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  successful_appeals['diff'] = successful_appeals.apply(


In [11]:
dfs_city = [
    count_appeals_city,
    count_parcels_city,
    pct_parcels_in_city,
    pct_appeals_in_city,
    pct_appeals_over_pct_parcels,
    count_unique_parcels_filing_appeal_city,
    pct_filing_appeal_of_fulton,
    pct_filing_appeal_by_city,
    success_count_city,
    success_rate_city,
    avg_appeal_change_city,
    pct_avg_appeal_change_city,
    med_appeal_change_city,
    sum_appealed_city,
    lost_revenue_city
]

df_names_city = [
    'count_appeals_city',
    'count_parcels_city',
    'pct_parcels_in_city',
    'pct_appeals_in_city',
    'pct_appeals_over_pct_parcels',
    'count_unique_parcels_filing_appeal_city',
    'pct_filing_appeal_of_fulton',
    'pct_filing_appeal_by_city',
    'success_count_city',
    'success_rate_city',
    'avg_appeal_change_city',
    'pct_avg_appeal_change_city',
    'med_appeal_change_city',
    'sum_appealed_city',
    'lost_revenue_city'
]

for i in range(1, 11):
    dfs_city.append(percent_filing_geq_than_n_appeals[i])
    df_names_city.append(f'percent_filing_geq_than_{i}_appeals')

by_city = pd.concat([pd.DataFrame(df) for df in dfs_city], axis=1)
by_city.columns = df_names_city

geq_n_appeals = pd.DataFrame({
    "city": by_city.index.unique()
}).merge(
    pd.DataFrame({'geq_n_appeals': [i for i in range(1, 11)]}),
    how='cross'
)
geq_n_appeals['appeals'] = geq_n_appeals.apply(
    lambda row: percent_filing_geq_than_n_appeals[row['geq_n_appeals']].loc[row['city']],
    axis=1
)

by_city = by_city.pivot_table(
    index='City',
    margins=True,
    margins_name='total',
    aggfunc=sum
)

C:\Users\Nick\AppData\Local\Temp\ipykernel_4236\536402533.py:55: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  by_city = by_city.pivot_table(


In [12]:
# Save table for animated map
geq_n_appeals.to_csv('output/geq_n_appeals.csv')

In [13]:
# Save table
by_city.to_csv('output/by_city.csv')
by_city

,avg_appeal_change_city,count_appeals_city,count_parcels_city,count_unique_parcels_filing_appeal_city,lost_revenue_city,med_appeal_change_city,pct_appeals_in_city,pct_appeals_over_pct_parcels,pct_avg_appeal_change_city,pct_filing_appeal_by_city,pct_filing_appeal_of_fulton,pct_parcels_in_city,percent_filing_geq_than_10_appeals,percent_filing_geq_than_1_appeals,percent_filing_geq_than_2_appeals,percent_filing_geq_than_3_appeals,percent_filing_geq_than_4_appeals,percent_filing_geq_than_5_appeals,percent_filing_geq_than_6_appeals,percent_filing_geq_than_7_appeals,percent_filing_geq_than_8_appeals,percent_filing_geq_than_9_appeals,success_count_city,success_rate_city,sum_appealed_city
City,,,,,,,,,,,,,,,,,,,,,,,,,
Alpharetta,157533.27,13749,21349,7235,5743789.18,17733.42,5.04,0.87,17.37,33.89,1.97,5.81,0.02,33.89,14.78,7.43,4.02,2.27,1.08,0.60,0.20,0.10,8440,61.39,1329580829.44
Atlanta,150815.34,137249,155640,63244,53446976.06,18411.64,50.34,1.19,21.61,40.63,17.21,42.34,0.03,40.63,21.97,12.21,6.77,3.55,1.84,0.79,0.30,0.10,82034,59.77,12371985199.26
Chattahoochee Hills,28610.62,2560,2833,1081,209498.37,10036.73,0.94,1.22,49.69,38.16,0.29,0.77,0.00,38.16,22.13,14.12,9.60,5.01,0.95,0.25,0.11,0.04,1695,66.21,48494992.80
College Park,171235.57,2231,4264,1082,918754.18,18035.06,0.82,0.71,30.83,25.38,0.29,1.16,0.00,25.38,12.80,6.80,4.15,1.85,0.96,0.26,0.09,0.02,1242,55.67,212674578.04
East Point,71678.85,7105,12940,3801,1123110.09,7766.90,2.61,0.74,29.39,29.37,1.03,3.52,0.01,29.37,13.71,6.30,2.89,1.55,0.70,0.26,0.09,0.04,3627,51.05,259979188.60
Fairburn,58224.83,4918,5951,2018,817728.11,5598.64,1.80,1.11,50.81,33.91,0.55,1.62,0.00,33.91,20.57,14.22,7.75,3.65,1.75,0.64,0.15,0.02,3251,66.10,189288913.30
Hapeville,98225.32,2187,2583,1051,566909.39,9196.64,0.80,1.14,25.71,40.69,0.29,0.70,0.04,40.69,21.53,11.73,6.12,2.71,1.05,0.54,0.19,0.08,1336,61.09,131229024.77
Johns Creek,74284.57,14133,26674,7928,2476136.54,14842.11,5.18,0.71,15.24,29.72,2.16,7.26,0.01,29.72,12.60,5.60,2.70,1.35,0.57,0.28,0.12,0.04,7716,54.60,573179753.80
Milton,55110.65,8610,13671,4897,1122537.76,19711.23,3.16,0.85,20.58,35.82,1.33,3.72,0.01,35.82,15.62,6.66,2.79,1.24,0.51,0.23,0.07,0.01,4715,54.76,259846704.51


## Generate table for metrics by year

- Count filing
- Percent filing
- Count success
- Percent success
- Sum of appealed amount
- Lost revenue

In [14]:
count_filing_yr = all_appeals.groupby('TAXYR')['ParcelID'].count()
percent_filing_yr = count_filing_yr / count_unique_parcels * 100

count_success_yr = successful_appeals.groupby('TAXYR')['ParcelID'].count()
pct_success_yr = count_success_yr / count_filing_yr * 100

sum_appealed_yr = successful_appeals.groupby('TAXYR')['diff'].agg('sum')

# Lost revenue, multiply 40% of assessed value by conservative 1.08% millage rate, or 0.00432
# TODO run this by someone
lost_revenue_yr = successful_appeals.groupby('TAXYR')['diff'].agg('sum') * 0.00432

In [15]:
dfs_yr = [
    count_filing_yr,
    percent_filing_yr,
    count_success_yr,
    pct_success_yr,
    sum_appealed_yr,
    lost_revenue_yr
]

df_names_yr = [
    'count_filing_yr',
    'percent_filing_yr',
    'count_success_yr',
    'pct_success_yr',
    'sum_appealed_yr',
    'lost_revenue_yr'
]

by_year = pd.concat([pd.DataFrame(df) for df in dfs_yr], axis=1)
by_year.columns = df_names_yr

by_year = by_year.pivot_table(
    index='TAXYR',
    margins=True,
    margins_name='total',
    aggfunc=sum
)

C:\Users\Nick\AppData\Local\Temp\ipykernel_4236\3840738769.py:22: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  by_year = by_year.pivot_table(


In [16]:
# Save table
by_year.to_csv('output/by_year.csv')
by_year

,count_filing_yr,count_success_yr,lost_revenue_yr,pct_success_yr,percent_filing_yr,sum_appealed_yr
TAXYR,,,,,,
2011.0,36241,22537,8588969.13,62.19,9.86,1988187298.80
2012.0,26572,20868,5576922.72,78.53,7.23,1290954333.35
2013.0,20306,15104,4315301.67,74.38,5.52,998912423.49
2014.0,18672,13318,3719158.48,71.33,5.08,860916314.30
2015.0,17745,11323,6165315.49,63.81,4.83,1427156364.52
2016.0,9541,5164,3796330.33,54.12,2.60,878780169.43
2017.0,18456,6719,4903668.14,36.41,5.02,1135108364.87
2018.0,39766,21650,11191325.53,54.44,10.82,2590584613.89
2019.0,23894,14603,10398074.69,61.12,6.50,2406961732.87


## Generate table for metrics by individual parcel

- Count of appeals per parcel
- Total sum appealed over all years
- Percent reduction over all years
- Lost revenue

In [17]:
count_appeals_per_parcel = all_appeals.groupby('ParcelID')['ParcelID'].count()
sum_appealed_per_parcel = successful_appeals.groupby('ParcelID')['diff'].agg('sum')
pct_reduction_per_parcel = sum_appealed_per_parcel / all_appeals.groupby('ParcelID')['NOTICE ASSESSMENT'].agg('sum') * 100
# Lost revenue, multiply 40% of assessed value by conservative 1.08% millage rate, or 0.00432
# TODO run this by someone
lost_revenue_per_parcel = successful_appeals.groupby('ParcelID')['diff'].agg('sum') * 0.00432

In [18]:
dfs_parcel = [
    count_appeals_per_parcel,
    sum_appealed_per_parcel,
    pct_reduction_per_parcel,
    lost_revenue_per_parcel
]

df_names_parcel = [
    'count_appeals_per_parcel',
    'sum_appealed_per_parcel',
    'pct_reduction_per_parcel',
    'lost_revenue_per_parcel'
]

by_parcel = pd.concat([pd.DataFrame(df) for df in dfs_parcel], axis=1)
by_parcel.columns = df_names_parcel

by_parcel = by_parcel.pivot_table(
    index='ParcelID',
    margins=True,
    margins_name='total',
    aggfunc=sum
)

C:\Users\Nick\AppData\Local\Temp\ipykernel_4236\495696815.py:18: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  by_parcel = by_parcel.pivot_table(


In [19]:
# Merge to Fulton parcels for geocode
by_parcel = by_parcel.merge(
    fulton_parcels,
    on='ParcelID',
    how='left'
)

In [20]:
# Save table
by_parcel.to_csv('output/by_parcel.csv')
by_parcel.head(3)

,ParcelID,count_appeals_per_parcel,lost_revenue_per_parcel,pct_reduction_per_parcel,sum_appealed_per_parcel,OBJECTID,TaxYear,Address,AddrNumber,AddrPreDir,AddrStreet,AddrSuffix,AddrPosDir,AddrUntTyp,AddrUnit,Owner,OwnerAddr1,OwnerAddr2,TaxDist,LUCode,ClassCode,ExCode,LivUnits,LandAcres,NbrHood,Subdiv,SubdivNum,SubdivLot,SubdivBlck,FeatureID,Shape__Area,Shape__Length,index_right,Name,WKT
0,06 0310 LL0490,3,314.45,7.03,72789.55,165522.00,2024.00,3916 DAHLWINY CT,3916,None,DAHLWINY,CT,None,None,None,HALDEMAN MAUREEN E & LARRY W,3916 DAHLWINY CT,SANDY SPRINGS GA 30350,59,101,R3,HF01S,1.00,0.35,2116,None,24400,3,None,06 0310 LL0490,14966.93,494.24,7.00,Sandy Springs,POINT (-84.27891 33.95790)
1,06 0310 LL0516,1,0.00,0.00,0.00,165535.00,2024.00,3924 DAHLWINY CT,3924,None,DAHLWINY,CT,None,None,None,TOTH WARREN & WENDY C,3924 DAHLWINY CT,SANDY SPRINGS GA 30350,59,101,R3,HF01,1.00,0.34,2116,None,24400,5,None,06 0310 LL0516,14915.18,499.12,7.00,Sandy Springs,POINT (-84.27836 33.95816)
2,06 0310 LL0524,1,0.00,0.00,0.00,165521.00,2024.00,3928 DAHLWINY CT,3928,None,DAHLWINY,CT,None,None,None,MARK & SARAH MOORE LIVING TRUST THE,3928 DAHLWINY CT,SANDY SPRINGS GA 30350,59,101,R3,HF01,1.00,0.34,2116,None,24400,6,None,06 0310 LL0524,15511.39,507.05,7.00,Sandy Springs,POINT (-84.27807 33.95830)
